# <center>Projeto Big Data</center> 
&nbsp;
#### **<center>Otimização da frota de ônibus turísticos</center>**
<br />
<br />
<br />
<br />
<center>Daniel Castro</center>
&nbsp;
<center>Theo Barbara</center>
<br />
<br />
<center>São Paulo | Nov 2022</center>
<br />

### Introdução 
<br />
<br />
    Este projeto tem como intuito a análise das rotas mais procuradas ao longo do ano a fim de possibilitar a alocação mais eficaz e lucrativa das frotas de ônibus. Ao ter conhecimento das rotas mais buscadas pelos passageiros é possível transferir ônibus que levariam bem menos passageiros para estas rotas com maior demanda, possibilitando uma maior captura de passageiros e consequentemente um lucro maior. Foram utilizadas para este estudo as bases de dados da Agência Nacional de Transportes Terrestres (ANTT), o link para o acesso às bases segue abaixo: 
<br />
<br />

[Base de dados ANTT](https://dados.antt.gov.br/dataset/monitriip-bilhetes-de-passagem)

<br />
<br />

### Organização dos dados
<br />
<br />
    Ao entrar no site da ANTT você vai perceber que os dados estão separados por mês, portanto para ter um volume de dados adequado para a análise foi preciso baixar alguns meses. Estes arquivos CSV de cada mês foram alocados em uma pasta e compactados para diminuir ligeiramente o tamanho do arquivo de trabalho.
<br />
<br />
    Para trabalhar com este arquivo ZIP com todos os meses a biblioteca ZipFile foi utilizada. O procedimento de análise dos dados foi basicamente abrir o arquivo ZIP, percorrer todos os arquivos que o compõem e logo após uní-los em um arquivo só.
<br />
<br />

In [1]:
from zipfile import ZipFile
import pandas as pd
import os
  
# Definindo o nome do arquivo de análise e seu caminho.
file_name = "Bilhetes.zip"
path = "C:/Users/danie/Desktop/Estudo/Eletivas/Big Data para dados públicos/Projeto-Big-Data/"
    
# Abrindo a arquivo zip com as informações de todos os meses
with ZipFile(os.path.join(path+file_name), 'r') as zip:
    # Printando o nome de todos os arquivo dentro do zip
    zip.printdir()  
    # Extraindo os arquivos para o diretório de trabalho
    zip.extractall()

File Name                                             Modified             Size
venda_passagem_01_2022.csv                     2022-10-22 16:20:22     17167260
venda_passagem_02_2022.csv                     2022-10-22 16:19:24     16620848
venda_passagem_03_2022.csv                     2022-10-22 16:18:58     17019407
venda_passagem_04_2022.csv                     2022-10-22 16:18:12      8500532
venda_passagem_05_2022.csv                     2022-10-22 16:17:38     16775582
venda_passagem_06_2022.csv                     2022-10-22 16:15:48     16292509
venda_passagem_07_2022.csv                     2022-10-22 16:00:36     19957908
venda_passagem_09_2021.csv                     2022-10-22 16:22:58     15840935
venda_passagem_09_2022.csv                     2022-10-22 15:54:58     17361183
venda_passagem_10_2021.csv                     2022-10-22 16:22:22     16216048
venda_passagem_11_2021.csv                     2022-10-22 16:21:48     16376100
venda_passagem_12_2021.csv              

In [2]:
# Criando o Data Frame de análise em branco e depois fazendo a inserção das informações de todos
# os meses
print('Construindo o arquivo...')
bilhetes = pd.DataFrame()
for file in zip.namelist():
    bilheteMes = pd.read_csv(file, sep=";", encoding="latin-1")
    try:
        bilhetes = pd.concat([bilhetes, bilheteMes], ignore_index=True)
        # Mostrando a evolução do tamanho da tabela 
        print(f"'{len(bilhetes)}' linhas.") 
    except Exception as error:
        print(error)
print(f"Finalizado, o arquivo possui '{len(bilhetes)}' linhas!'")

Construindo o arquivo...
'132157' linhas.
'260331' linhas.
'391086' linhas.
'457984' linhas.
'586954' linhas.
'713771' linhas.
'867262' linhas.
'988633' linhas.
'1124196' linhas.
'1248443' linhas.
'1374418' linhas.
'1527032' linhas.
Finalizado, o arquivo possui '1527032' linhas!'


In [5]:
# Limpando linhas com valores NaN
if len(bilhetes[bilhetes.isna().any(axis=1)]) != 0:
    print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN que"+
          "devem ser apagadas")
    print("Removendo linhas NaN...")
    bilhetes.dropna(inplace=True)
print(f"O arquivo contém '{len(bilhetes[bilhetes.isna().any(axis=1)])}' linhas com NaN!")

O arquivo contém '0' linhas com NaN!


In [6]:
bilhetes.sample(5)

,mes_emissao_bilhete,mes_viagem,ponto_origem_viagem,ponto_destino_viagem,tipo_servico,tipo_gratuidade,media_valor_total,dp_valor_total,quantidade_bilhetes
608032,06/2022,07/2022,GOIANIA/GO,SEABRA/BA,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",279.85,10.88,18
710204,06/2022,07/2022,GOIANIA/GO,BRASILIA/DF,Leito com ar condicionado,Tarifa Normal - sem desconto,48.79,20.20,5
1000445,set/22,out/22,AMERICANA/SP,CURITIBA/PR,Semileito,Tarifa Normal - sem desconto,162.15,37.27,16
1245920,10/2021,10/2021,JATAI/GO,ILHA SOLTEIRA/SP,Executivo,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",114.01,36.14,11
1065596,set/22,set/22,VALPARAISO DE GOIAS/GO,JUIZ DE FORA/MG,Leito com ar condicionado,"Tarifa Promocional - Parágrafo 3º, art. 27 do ...",423.00,0.00,2


Perceba que nas colunas "_mes_emissao_bilhete_" e "_mes_viagem_" a informação da data pode ser apresentada em dois formatos diferentes: totalmente numérico, ou seja, **09/2022**, ou então utilizando a sigla do mês e uma abreviação do ano, ou seja, **set/22**. 

<br />
É preciso unificar a apresentação das informações para que a análise não seja atrapalhada por esta divergência de formato. O formato escolhido foi o númerico, logo abaixo será feita a conversão das informações das colunas.

<br />
<br />

**É imprescindível que as informações de data contenham apenas mês e ano.**

In [7]:
def conversorData(data:str):
    """
    Converte as datas que estão em um foramto diferente do esperado.
    :param data: data a ser convertida
    return: 
    - Caso data seja passível de conversão, retorna a data convertida no formato MM/AAAA.
    
    - Caso contrário, retorna "Informação inválida" 
    """        
    
    mes, ano = data.split('/')
    try:
        int(mes)
    except Exception as error:
        if type(error) is ValueError: # Significa que existe alguma letra
            mes = mes.lower()
            # Faz a respetiva atribuição numérica da sigla do mês da data
            if mes == 'jan':
                mes = '01'
            elif mes == 'fev':
                mes = '02'
            elif mes == 'mar':
                mes = '03'
            elif mes == 'abr':
                mes = '04'
            elif mes == 'mai':
                mes = '05'
            elif mes == 'jun':
                mes = '06'
            elif mes == 'jul':
                mes = '07'
            elif mes == 'ago':
                mes = '08'
            elif mes == 'set':
                mes = '09'
            elif mes == 'out':
                mes = '10'
            elif mes == 'nov':
                mes = '11'
            elif mes == 'dez':
                mes = '12'
            else:
                return 'Informação inválida' 
    else:
        if int(mes) not in range(1,13): # o mês não é válida
            return 'Informação inválida' 
    finally:
        if len(ano) == 2: # A data está no formato MM/AA e deve ser convertida para MM/AAAA
            if int(ano) in range(0,25):
                ano = '20'+ano
            else:
                ano = '19'+ano
        return mes+'/'+ano

In [8]:
# Conversão das colunas de data
# A linha abaixo possui o formato errado de data.
print(f"Formato original da data: '{bilhetes.loc[1019501, 'mes_viagem']}'")
print('Convertendo...')
bilhetes['mes_viagem'] = bilhetes['mes_viagem'].apply(lambda x: conversorData(x))
bilhetes['mes_emissao_bilhete'] = bilhetes['mes_emissao_bilhete'].apply(lambda x: conversorData(x))
print(f"Formato da data após a conversão: '{bilhetes.loc[1019501, 'mes_viagem']}'")

Formato original da data: 'set/22'
Convertendo...
Formato da data após a conversão: '09/2022'


### Análise preliminar
<br />
Para começar é interessante observar graficamente para um mês qualquer, qual a rota que possui a maior quantidade de bilhetes e consequentemente a maior quantidade de passageiros. 

Vou utilizar o mês do meu aniversário só por coincidência.

In [65]:
import matplotlib.pyplot as plt
import plotly.express as px

BilhetesABR = bilhetes[(bilhetes['mes_viagem'].str[0:2] == '04')]
df = BilhetesABR.groupby(['ponto_origem_viagem']).count()
df = df.loc[df['mes_emissao_bilhete']>500].sort_values(by=['mes_viagem'], ascending=False)

px.bar(df, y='mes_viagem',
       labels={'mes_viagem': 'N° bilhetes emitidos', 'ponto_origem_viagem': 'Cidade de partida da viagem'},
       title='Cidades com maior número de passageiros emigrantes em ABR/2022',
       template='plotly_white')                                              

In [66]:
df2 = BilhetesABR.groupby(['ponto_destino_viagem']).count()
df2 = df2.loc[df2['mes_emissao_bilhete']>500].sort_values(by=['mes_viagem'], ascending=False)
px.bar(df2, y='mes_viagem',
       labels={'mes_viagem': 'N° de bilhetes', 'ponto_destino_viagem': 'Cidade de destino da viagem'},
       title='Cidades com maior número de passageiros imigrantes em ABR/2022',
       template='plotly_white')

Perceba que tanto no ponto de partida, quanto no ponto de destino as cinco cidades predominantes são:

<br />

 *  São Paulo (SP)
 *  Rio de Janeiro (RJ)
 *  Goiânia (GO)
 *  Brasília (DF)
 *  Curitiba (PR)
 
<br />

Com certeza estas são cidades boas para se inserir em suas rotas de viagem. 

Mas existe também duas outras perguntas: 

<br />

 *  "Estas pessoas que saem de São Paulo, para onde elas vão?"

<br />

 *  "E as pessoas que vêm para São Paulo, de onde elas partem?"
 
<br />

Para responder a estas perguntas vamos fazer uma cross table e ver o resultado.

In [70]:
rotas = pd.crosstab(BilhetesABR['ponto_origem_viagem'],BilhetesABR['ponto_destino_viagem'])
rotasSP=rotas.loc[:,["SAO PAULO/SP"]].sort_values(by=['SAO PAULO/SP'], ascending = False)
rotasSP = rotasSP.loc[rotasSP['SAO PAULO/SP'] != 0] # Retirando as cidades que não possuem bilhetes

In [71]:
px.bar(rotasSP, title='Rotas com sentido a São Paulo em ABR/2022',
       labels={'value':'N° de bilhetes', 'ponto_origem_viagem': 'Cidade de origem'},
       template='plotly_white')

As 10 cidades que mais possuem passageiros em direção a São Paulo em abril são:

In [75]:
rotasSP.head(10)

ponto_origem_viagem,SAO PAULO/SP
ponto_destino_viagem,
CURITIBA/PR,35
LONDRINA/PR,33
RIO DE JANEIRO/RJ,33
BELO HORIZONTE/MG,32
MARINGA/PR,31
VITORIA DA CONQUISTA/BA,30
BRASILIA/DF,29
GOIANIA/GO,28
FOZ DO IGUACU/PR,26


In [76]:
rotas = pd.crosstab(BilhetesABR['ponto_destino_viagem'],BilhetesABR['ponto_origem_viagem'])
rotasSP=rotas.loc[:,["SAO PAULO/SP"]].sort_values(by=['SAO PAULO/SP'], ascending = False)
rotasSP = rotasSP.loc[rotasSP['SAO PAULO/SP'] != 0] # Retirando as cidades que não possuem bilhetes
px.bar(rotasSP, title='Rotas saindo de São Paulo em ABR/2022',
       labels={'value':'N° de bilhetes', 'ponto_origem_viagem': 'Cidade de origem'},
       template='plotly_white')

As 10 cidades que os passageiros de São Paulo mais viajam em abril são:

In [74]:
rotasSP.head(10)

ponto_origem_viagem,SAO PAULO/SP
ponto_destino_viagem,
CURITIBA/PR,35
LONDRINA/PR,33
RIO DE JANEIRO/RJ,33
BELO HORIZONTE/MG,32
MARINGA/PR,31
VITORIA DA CONQUISTA/BA,30
BRASILIA/DF,29
GOIANIA/GO,28
FOZ DO IGUACU/PR,26
